In [1]:
import numpy as np
import tensorflow as tf
import gc

2023-02-11 14:44:50.020359: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_physical_devices('GPU')

Num GPUs Available:  4


2023-02-11 14:44:50.871146: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-02-11 14:44:50.872133: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-02-11 14:44:50.916210: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:18:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2023-02-11 14:44:50.917079: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:51:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2023-02-11 14:44:50.917837: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:8b:00.0 na

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
#   try:
#     tf.config.set_visible_devices(gpus[1], 'GPU')
#     logical_gpus = tf.config.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
#   except RuntimeError as e:
#     # Visible devices must be set before GPUs have been initialized
#     print(e)

In [5]:
# load MNIST data.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# Preprocessing.
x_train = x_train / 255.0
x_test = x_test / 255.0
shuffle_index = np.random.permutation(len(x_train))
x_train, y_train = x_train[shuffle_index], y_train[shuffle_index]
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)
# Track the data type.
dataType = x_train.dtype
print(f"Data type: {dataType}")
labelType = y_test.dtype
print(f"Data type: {labelType}")

Data type: float64
Data type: float32


In [6]:
# Instantiate the training dataset.
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

In [7]:
# Create the model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense


In [8]:
# model_ref = Sequential()
# model_ref.add(Dense(units=32, activation='relu'))
# model_ref.add(Dense(units=32, activation='relu'))
# model_ref.add(Dense(units=10, activation='softmax'))

In [ ]:
from tensorflow import keras
# Create an instance of the model   
inputs = keras.Input(shape=(784,), name="digits")
x1 = keras.layers.Dense(32, activation="relu", kernel_initializer="uniform",
    bias_initializer="uniform")(inputs)
print(1)
x2 = keras.layers.Dense(32, activation="relu", kernel_initializer="uniform",
    bias_initializer="uniform")(x1)
print(2)
# x3 = keras.layers.Dense(32, activation="relu", kernel_initializer="uniform",bias_initializer="uniform")(x2)
outputs = keras.layers.Dense(10, name="predictions",kernel_initializer="uniform",
    bias_initializer="uniform")(x2)
print(3)
model_ref = keras.Model(inputs=inputs, outputs=outputs)

In [8]:
from tensorflow import keras

In [9]:
inputs = keras.Input(shape=(784,), name="digits")

In [ ]:
x1 = keras.layers.Dense(32, activation="relu", kernel_initializer="uniform",
    bias_initializer="uniform")(inputs)